In [ ]:
import cv2
import numpy as np
import insightface
from insightface.app import FaceAnalysis
import pickle
import os


FACE_DB = "face_database.pkl"

def load_face_database():
    if os.path.exists(FACE_DB):
        with open(FACE_DB, "rb") as f:
            return pickle.load(f)
    return {}

def save_face_database(db):
    with open(FACE_DB, "wb") as f:
        pickle.dump(db, f)

app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

app.prepare(ctx_id=0)


face_db = load_face_database()

def register_face(name, embedding):
    face_db[name] = embedding
    save_face_database(face_db)
    print(f"چهره {name} ثبت شد!")

def recognize_face(embedding):
    best_match = None
    min_dist = float("inf")
    for name, db_embedding in face_db.items():
        dist = np.linalg.norm(embedding - db_embedding)
        if dist < min_dist:
            min_dist = dist
            best_match = name
    return best_match if min_dist < 1.0 else "unknow"


cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    faces = app.get(frame)
    for face in faces:
        x1, y1, x2, y2 = map(int, face.bbox)
        embedding = face.embedding


        name = recognize_face(embedding)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("Face Recognition", frame)

    key = cv2.waitKey(1)
    print("Detected Faces:", faces)
    if key == ord('q'):
        break
    elif key == ord('r'):
        name = input("نام شخص را وارد کنید: ")
        if faces:
            register_face(name, faces[0].embedding)

cap.release()
cv2.destroyAllWindows()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/sina/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size